In [2]:
import pandas as pd
import tensorflow as tf

2025-04-03 00:48:51.138327: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 00:48:51.151032: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743621531.165021   16044 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743621531.168247   16044 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 00:48:51.183289: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [22]:
df = pd.read_pickle("all_data.pkl")

In [4]:
df.head()

,sans_sent,kon_sent
0,<kn> अहं प्रतिदिनं प्रभाते जागामि।,[start] हांव रोज सकाळीं उठतां। ...
1,<kn> मम गृहं नदीतीरे अस्ति।,[start] म्हजें घर न्हंयकडेल्या आहे। ...
2,<kn> सूर्यः पूर्वस्यां दिशि उदति।,[start] सूर्य पूवरे कडेन उगता। [...
3,<kn> वयं संनादति क्षेत्रे क्रीडामः।,[start] आम्ही एकठांय शेतांत खेळतांव। ...
4,<kn> माता भोजनं पचति।,[start] आजी जेवण तयार करता। [end]


In [23]:
import random
n = random.randint(0,len(df))

df['sans_sent'][n],df['kon_sent'][n]

('<kn> अधिकमलप्रवृत्ति उदरविषाणोः लक्षणम्।',
 '[start] वारंवार जुलाब व्हायरल इंफेक्शनचं लक्षण। [end]')

In [5]:
len(df)

1835

In [6]:
import sentencepiece as spm
sanskrit_tokenizer = spm.SentencePieceProcessor()
sanskrit_tokenizer.load("./model/sanskrit_tokenizer_2.model")

True

In [7]:
hindi_tokenizer = spm.SentencePieceProcessor()
hindi_tokenizer.load("./model/konkani_tokenizer.model")

True

In [8]:
import numpy as np
def pad_same_lenth(sent,tokenize,max_len):
    sent = tokenize.encode(sent)
    if len(sent) > max_len:
        return tf.convert_to_tensor(sent[:max_len],dtype="int32")
    else:
        return tf.convert_to_tensor(np.pad(sent, (0, max_len - len(sent))),dtype="int32")

In [9]:
max_len = 20
df["sans_sent"] = df["sans_sent"].apply(lambda x:pad_same_lenth(x,sanskrit_tokenizer,max_len))
df["kon_sent"] = df["kon_sent"].apply(lambda x: pad_same_lenth(x, hindi_tokenizer, max_len+1))

I0000 00:00:1743621587.338611   16044 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [10]:
len(df["sans_sent"][38]),len(df["kon_sent"][78])

(20, 21)

In [12]:
val_ds = df.iloc[:1000]
train_ds = df.iloc[1000:]

In [14]:
batch_size = 32


def format_dataset(sans, hindi):
    return (
        {
            "english": sans,
            "spanish": hindi[:, :-1],
        },
        hindi[:, 1:],
    )


def make_dataset(pairs,src,trg):
    # eng_texts, spa_texts = zip(*pairs)
    src_text = pairs[src].to_list()
    targ_text = pairs[trg].to_list()
    dataset = tf.data.Dataset.from_tensor_slices((src_text, targ_text))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

src = "sans_sent"
targ = "kon_sent"

train_ds = make_dataset(train_ds,src,targ)
val_ds = make_dataset(val_ds,src,targ)

In [27]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)


2025-04-02 21:17:43.633484: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-04-02 21:17:43.634099: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "dense_dim": self.dense_dim,
            }
        )
        return config

In [16]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "dense_dim": self.dense_dim,
            }
        )
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [41]:
# @tf.keras.saving.register_keras_serializable()
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    # def compute_mask(self, inputs, mask=None):
    #     return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update(
            {
                "output_dim": self.output_dim,
                "sequence_length": self.sequence_length,
                "input_dim": self.input_dim,
            }
        )
        return config

In [44]:
embed_dim = 128
dense_dim = 512
num_heads = 4
sequence_length = 20
vocab_size = 3940

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [45]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    "best_model.keras", monitor="val_loss", save_best_only=True, mode="min", verbose=1
)

In [46]:
transformer.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
hist = transformer.fit(train_ds, epochs=4, validation_data=val_ds,callbacks=[checkpoint])

Epoch 1/4


W0000 00:00:1743623043.863053   17058 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2025-04-03 01:14:04.978378: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_14_0', 768 bytes spill stores, 720 bytes spill loads

2025-04-03 01:14:04.986336: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_14', 188 bytes spill stores, 188 bytes spill loads

2025-04-03 01:14:05.097183: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_14_0', 228 bytes spill stores, 228 bytes spill loads



12/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4737 - loss: 6.6746   

2025-04-03 01:14:08.533302: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'copy_fusion', 192 bytes spill stores, 192 bytes spill loads
ptxas warning : Registers are spilled to local memory in function '__cuda_sm3x_div_rn_noftz_f32_slowpath', 44 bytes spill stores, 44 bytes spill loads

W0000 00:00:1743623048.901483   17059 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.5670 - loss: 5.8824

W0000 00:00:1743623053.227728   17059 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1743623054.367984   17062 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2025-04-03 01:14:15.049398: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_14_0', 756 bytes spill stores, 444 bytes spill loads




Epoch 1: val_loss improved from inf to 2.93786, saving model to best_model.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 15s 278ms/step - accuracy: 0.5798 - loss: 5.7280 - val_accuracy: 0.6807 - val_loss: 2.9379
Epoch 2/4
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8053 - loss: 1.7103
Epoch 2: val_loss improved from 2.93786 to 2.33413, saving model to best_model.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8060 - loss: 1.6893 - val_accuracy: 0.7218 - val_loss: 2.3341
Epoch 3/4
24/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8324 - loss: 1.0417
Epoch 3: val_loss improved from 2.33413 to 2.20869, saving model to best_model.keras
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8323 - loss: 1.0454 - val_accuracy: 0.7342 - val_loss: 2.2087
Epoch 4/4
24/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8566 - loss: 0.8656
Epoch 4: val_loss did not improve from 2.20869
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8558 - loss: 0.8711 - val_accuracy: 0.7352 - v

In [123]:
transformer.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ english             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spanish             │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  5,125,120 │ english[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │  5,125,120 │ spanish[0][0]     │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_encode… │ (None, None, 256) │  3,155,456 │ positional_embed… │
│ (TransformerEncode… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_decode… │ (None, None, 256) │  5,259,520 │ positional_embed… │
│ (TransformerDecode… │                   │            │ transformer_enco… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, None, 256) │          0 │ transformer_deco… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, None,      │  5,140,000 │ dropout_11[0][0]  │
│                     │ 20000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,805,216 (90.81 MB)

 Trainable params: 23,805,216 (90.81 MB)

 Non-trainable params: 0 (0.00 B)

In [145]:
tf.convert_to_tensor(hindi_tokenizer.encode(["[start]"]))

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[19877,     3]], dtype=int32)>

In [48]:
import numpy as np

# spa_vocab = target_vectorization.get_vocabulary()
# spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence,transformer):
    hehe = sanskrit_tokenizer.encode(input_sentence)[:20]
    tokenized_input_sentence = tf.convert_to_tensor([hehe])
    # print(tokenized_input_sentence)
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = tf.convert_to_tensor([hindi_tokenizer.encode(decoded_sentence)[:][:-1]])
        # print([tokenized_input_sentence, tokenized_target_sentence])
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        print(sampled_token_index)
        sampled_token = hindi_tokenizer.decode(int(sampled_token_index))
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence


# test_eng_texts = [pair[0] for pair in test_pairs]
# for _ in range(20):
#     input_sentence = random.choice(test_eng_texts)
#     print("-")
#     print(input_sentence)
#     print(decode_sequence(input_sentence))

decode_sequence(
    "<kn> अधिकमलप्रवृत्ति उदरविषाणो लक्षणम्", transformer
)  # [start] Do you celebrate Easter in Brazil? [end]

3
51
4


'[start] [start] हांव [end]'

In [49]:
best_model = tf.keras.models.load_model("best_model.keras",custom_objects=transformer.get_config())

TypeError: <class 'keras.src.models.functional.Functional'> could not be deserialized properly. Please ensure that components that are Python object instances (layers, models, etc.) returned by `get_config()` are explicitly deserialized in the model's `from_config()` method.

config={'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {}, 'registered_name': 'Functional', 'build_config': {'input_shape': None}, 'compile_config': {'optimizer': {'module': 'keras.optimizers', 'class_name': 'Adam', 'config': {'name': 'adam', 'learning_rate': 0.0010000000474974513, 'weight_decay': None, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}, 'registered_name': None}, 'loss': 'sparse_categorical_crossentropy', 'loss_weights': None, 'metrics': ['accuracy'], 'w

In [29]:
a = transformer.get_config()

In [30]:
new_model = tf.keras.Model.from_config(a)

TypeError: Could not locate class 'PositionalEmbedding'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': None, 'class_name': 'PositionalEmbedding', 'config': {'name': 'positional_embedding', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'output_dim': 128, 'sequence_length': 20, 'input_dim': 3940}, 'registered_name': 'PositionalEmbedding', 'build_config': {'input_shape': (None, None)}, 'name': 'positional_embedding', 'inbound_nodes': [{'args': ({'class_name': '__keras_tensor__', 'config': {'shape': (None, None), 'dtype': 'int64', 'keras_history': ['english', 0, 0]}},), 'kwargs': {}}]}